# Mapping Molecules

In this notebook we will cover how to map molecules in different ways and look at some of the things we can do with them. In the first part of the tutorial we will look at different ways to map water molecules and in the second, we will extend this to a more complex molecule system, the room temperature ionic liquid, BMIM-BF4.

# Water Molecules

In the first part of this demonstration we will load data from a GROMACS simulation and therefore, we need to define a set of units and a file reader object to use. For this reason, we have changed the imports a little bit to keep the code to minimum.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import mdsuite as mds
import mdsuite.file_io.chemfiles_read
from mdsuite.utils import Units

from zinchub import DataHub
import shutil

import h5py as hf
import numpy as np

2022-02-22 14:17:01.857589: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-22 14:17:01.857640: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: tahr
2022-02-22 14:17:01.857649: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: tahr
2022-02-22 14:17:01.857778: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.103.1
2022-02-22 14:17:01.857812: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.103.1
2022-02-22 14:17:01.857820: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.103.1


### Loading the data

In this tutorial we are using 50 ns simulations of 14 water molecules in a continuum fluid performed with GROMACS. We will use pure atomistic naming as well as ligand naming, the topology files for which are contained on DataHub.

In [2]:
water = DataHub(url="https://github.com/zincware/DataHub/tree/main/Water_14_Gromacs")
water.get_file('./')
file_paths = [
        f for f in water.file_raw
    ]

### Project definition

Here we create the project and define some custom units used by GROMACS.

In [3]:
project = mds.Project("Mapping_Molecules")

gmx_units = Units(
        time=1e-12,
        length=1e-10,
        energy=1.6022e-19,
        NkTV2p=1.6021765e6,
        boltzmann=8.617343e-5,
        temperature=1,
        pressure=100000,
    )

2022-02-22 14:17:05,875 - INFO: Creating new project Mapping_Molecules


### Mapping molecules with SMILES

In this section we take a look at how one can map molecules using SMILES strings.

In [4]:
traj_path = file_paths[2]
topol_path = file_paths[0]

file_reader = mdsuite.file_io.chemfiles_read.ChemfilesRead(
    traj_file_path=traj_path, topol_file_path=topol_path
)

water_chemical = project.add_experiment(
    name="water_chemical",
    timestep=0.002,
    temperature=300.0,
    units=gmx_units,
    simulation_data=file_reader,
)
water_chemical.sample_rate = 5000
water_chemical.run.CoordinateUnwrapper()

2022-02-22 14:17:06,196 - INFO: Creating a new experiment!
2022-02-22 14:17:06,507 - WARNING: No GPUs detected, continuing without GPU support


100%|███████████████████████████████████| 1/1 [00:00<00:00,  1.54it/s]
2022-02-22 14:17:07.350626: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2022-02-22 14:17:07,371 - WARNING: No GPUs detected, continuing without GPU support


2022-02-22 14:17:07.432165: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
Applying transformation 'Unwrapped_Positions' to 'O': 100%|█| 1/1 [00:

2022-02-22 14:17:07,837 - WARNING: No GPUs detected, continuing without GPU support



Applying transformation 'Unwrapped_Positions' to 'H': 100%|█| 1/1 [00:


The first thing we need to do is define the molecule that will be mapped using the in-built MDSuite molecule data-class

In [5]:
chemical_water = mds.Molecule(
    name='water',
    smiles="[H]O[H]", 
    cutoff=1.7, 
    amount=14, 
    mol_pbc=True
)

In [6]:
water_chemical.run.MolecularMap(
    molecules=[chemical_water]
)

2022-02-22 14:17:08,140 - INFO: Building molecular graph from configuration for water


100%|████████████████████████████████| 42/42 [00:00<00:00, 105.73it/s]


2022-02-22 14:17:08,830 - INFO: Performing molecule number isomorphism test.
2022-02-22 14:17:08,831 - INFO: Amount of molecules test passed.
2022-02-22 14:17:08,832 - INFO: Performing group equality isomorphism test.
2022-02-22 14:17:08,833 - INFO: Group equality isomorphism test passed.
2022-02-22 14:17:08,871 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:17:08,913 - INFO: Mapping molecule graphs onto trajectory for water


100%|███████████████████████████████████| 1/1 [00:00<00:00,  6.57it/s]


2022-02-22 14:17:09,164 - WARNING: No GPUs detected, continuing without GPU support


Applying transformation 'Unwrapped_Positions' to 'water': 100%|█| 1/1 


In [7]:
water_chemical.molecules

{'water': {'n_particles': 14,
  'mass': 18.015,
  'groups': {'0': {'H': [0, 1], 'O': [0]},
   '1': {'H': [2, 3], 'O': [1]},
   '2': {'H': [4, 5], 'O': [2]},
   '3': {'H': [6, 7], 'O': [3]},
   '4': {'H': [8, 9], 'O': [4]},
   '5': {'H': [10, 11], 'O': [5]},
   '6': {'H': [12, 13], 'O': [6]},
   '7': {'H': [14, 15], 'O': [7]},
   '8': {'H': [16, 17], 'O': [8]},
   '9': {'H': [18, 19], 'O': [9]},
   '10': {'H': [20, 21], 'O': [10]},
   '11': {'H': [22, 23], 'O': [11]},
   '12': {'H': [24, 25], 'O': [12]},
   '13': {'H': [26, 27], 'O': [13]}}}}

### Mapping Molecules with a reference dict

If you do not have particles with chemical names but you nonetheless wish to construct groups out of particles, this can be achieved by using a reference dict.

In this example, we use the ligand naming from GROMACS to construct water molecules.

In [8]:
traj_path = file_paths[2]
topol_path = file_paths[1]

file_reader = mdsuite.file_io.chemfiles_read.ChemfilesRead(
    traj_file_path=traj_path, topol_file_path=topol_path
)

water_ligand = project.add_experiment(
    name="water_ligand",
    timestep=0.002,
    temperature=300.0,
    units=gmx_units,
    simulation_data=file_reader,
)
water_ligand.run.CoordinateUnwrapper()

2022-02-22 14:17:09,496 - INFO: Creating a new experiment!
2022-02-22 14:17:10,247 - WARNING: WARNING element OW has been assigned mass=0.0
2022-02-22 14:17:11,625 - WARNING: No GPUs detected, continuing without GPU support


100%|███████████████████████████████████| 1/1 [00:00<00:00,  1.25it/s]

2022-02-22 14:17:12,629 - WARNING: No GPUs detected, continuing without GPU support



Applying transformation 'Unwrapped_Positions' to 'OW': 100%|█| 1/1 [00

2022-02-22 14:17:12,825 - WARNING: No GPUs detected, continuing without GPU support



Applying transformation 'Unwrapped_Positions' to 'HW1': 100%|█| 1/1 [0


2022-02-22 14:17:13,002 - WARNING: No GPUs detected, continuing without GPU support


Applying transformation 'Unwrapped_Positions' to 'HW2': 100%|█| 1/1 [0


Keep in mind, as the particles are not named from the periodic tables, important properties such as mass will need to be filled in manually.

In [9]:
water_ligand.species['OW'].mass = [15.999]
water_ligand.species['HW1'].mass = [1.00784]
water_ligand.species['HW2'].mass = [1.00784]

In this case, the molecule will be defined a little bit differently.

In [10]:
ligand_water = mds.Molecule(
    name='water', 
    cutoff=1.7, 
    amount=14, 
    species_dict={"HW1": 1, "OW": 1, "HW2": 1},
    mol_pbc=True
)

In [11]:
water_ligand.run.MolecularMap(
    molecules=[ligand_water]
)

2022-02-22 14:17:13,206 - INFO: Building molecular graph from configuration for water


100%|████████████████████████████████| 42/42 [00:00<00:00, 489.33it/s]


2022-02-22 14:17:13,474 - INFO: Performing molecule number isomorphism test.
2022-02-22 14:17:13,475 - INFO: Amount of molecules test passed.
2022-02-22 14:17:13,476 - INFO: Performing group equality isomorphism test.
2022-02-22 14:17:13,476 - INFO: Group equality isomorphism test passed.
2022-02-22 14:17:13,517 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:17:13,555 - INFO: Mapping molecule graphs onto trajectory for water


100%|███████████████████████████████████| 1/1 [00:00<00:00,  6.32it/s]


2022-02-22 14:17:13,817 - WARNING: No GPUs detected, continuing without GPU support


Applying transformation 'Unwrapped_Positions' to 'water': 100%|█| 1/1 


In [12]:
water_ligand.molecules

{'water': {'n_particles': 14,
  'mass': 18.014680000000002,
  'groups': {'0': {'HW1': [0], 'OW': [0], 'HW2': [0]},
   '1': {'HW1': [1], 'OW': [1], 'HW2': [1]},
   '2': {'HW1': [2], 'OW': [2], 'HW2': [2]},
   '3': {'HW1': [3], 'OW': [3], 'HW2': [3]},
   '4': {'HW1': [4], 'OW': [4], 'HW2': [4]},
   '5': {'HW1': [5], 'OW': [5], 'HW2': [5]},
   '6': {'HW1': [6], 'OW': [6], 'HW2': [6]},
   '7': {'HW1': [7], 'OW': [7], 'HW2': [7]},
   '8': {'HW1': [8], 'OW': [8], 'HW2': [8]},
   '9': {'HW1': [9], 'OW': [9], 'HW2': [9]},
   '10': {'HW1': [10], 'OW': [10], 'HW2': [10]},
   '11': {'HW1': [11], 'OW': [11], 'HW2': [11]},
   '12': {'HW1': [12], 'OW': [12], 'HW2': [12]},
   '13': {'HW1': [13], 'OW': [13], 'HW2': [13]}}}}

### What information is stored?

So the molecule mapping itself was quick and easy, but what information has been stored along the way?

All meta-data about the molecules is stored in the experiment class under molecules. Let's take a look at what this contains.

In [13]:
water_chemical.molecules.keys()

dict_keys(['water'])

This dict will contain all of the molecules that have been mapped, but this is not the information about the molecules, for that, we need to look at the water molecule.

In [14]:
water_chemical.molecules['water'].keys()

dict_keys(['n_particles', 'mass', 'groups'])

Three of these are fairly trivial and we can look at them quickly, groups will require some more attention.

In [15]:
print(f"n_particles: {water_chemical.molecules['water']['n_particles']}")
print(f"mass: {water_chemical.molecules['water']['mass']}")

n_particles: 14
mass: 18.015


Now let's take a look at the groups key.

In [16]:
print(water_chemical.molecules['water']['groups'])

{'0': {'H': [0, 1], 'O': [0]}, '1': {'H': [2, 3], 'O': [1]}, '2': {'H': [4, 5], 'O': [2]}, '3': {'H': [6, 7], 'O': [3]}, '4': {'H': [8, 9], 'O': [4]}, '5': {'H': [10, 11], 'O': [5]}, '6': {'H': [12, 13], 'O': [6]}, '7': {'H': [14, 15], 'O': [7]}, '8': {'H': [16, 17], 'O': [8]}, '9': {'H': [18, 19], 'O': [9]}, '10': {'H': [20, 21], 'O': [10]}, '11': {'H': [22, 23], 'O': [11]}, '12': {'H': [24, 25], 'O': [12]}, '13': {'H': [26, 27], 'O': [13]}}


The groups key contains direct information about which atoms belong to which molecule, for example, the 10th molecule of water (id=9) consists of Hydrogen atoms 18 and 19 and oxygen atom 10.

In [17]:
print(water_chemical.molecules['water']['groups']['9'])

{'H': [18, 19], 'O': [9]}


With this information you can compute values, e.g. diffusion coefficients with only the atoms belonging to a single molecule using the atom_select arguments in the calculator.

### Analysis with molecules

Now that we have seen how we can build molecules and what information this gives is, let's look at what we can analyse using them.

#### Angular Distribution Functions (ADFs)

First things first, let's confirm we are working with water by looking at the angular distribution function of the atoms.

In [18]:
water_chemical.run.AngularDistributionFunction(
    number_of_configurations=5000, number_of_bins=500, norm_power=8
)

2022-02-22 14:17:14,151 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:17:14,181 - WARNING: No GPUs detected, continuing without GPU support


100%|███████████████████████████████████| 1/1 [00:36<00:00, 36.29s/it]


Loading BokehJS ...

Exp1_Angular_Distribution_Function_1

Looking at the O_H_H ADF in he top right we see a strong max peak at 109.591 degrees corresponding well with the bond angle of an SPCE model (109.47) as was used in the simulation. It is also worth noting that the oxygen triplet angle looks similar to that measured in QM and experimental studies.

When we want to study the molecular ADF we have two choices, we can either pass it as a species argument to the calculator if only one is desired, we we can call the calculator with the `molecules=True` keyword as we will do here.

In [19]:
water_chemical.run.AngularDistributionFunction(
    molecules=True, number_of_configurations=3000, number_of_bins=500, norm_power=8
)

2022-02-22 14:17:51,225 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:17:51,264 - WARNING: No GPUs detected, continuing without GPU support


100%|███████████████████████████████████| 1/1 [00:01<00:00,  1.01s/it]


Loading BokehJS ...

Exp1_Angular_Distribution_Function_2

In this case we have increased the norm power to suppress the noise floor and highlight only the most dominant peaks.

In the water molecule ADF it we do not see any clear stacking or structure suggesting there is not special organization of the molecules in these simulations.

#### Radial Distribution Functions (RDFs)

Now let's look at the radial structure and distribution of particles in space of both the atomistic system and the molecules. This is where molecule mapping can be very helpful as often we are more interested in the positions of the molecules themselves and not necessarily those of the atoms.

In [20]:
water_chemical.run.RadialDistributionFunction(
    number_of_configurations=4000, start=100, stop=5100, number_of_bins=500
)

2022-02-22 14:17:52,612 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:17:52,650 - WARNING: No GPUs detected, continuing without GPU support


Running mini batch loop 3 / 3: 100%|████| 1/1 [00:00<00:00, 15.27it/s]


Loading BokehJS ...

Exp1_Radial_Distribution_Function_3

In the case of the hydrogen-hydrogen and the oxygen-hydrogen we can see clear peaks where the bond distance is fixed. Using the cursor to hover over the points in the plot we can identify a bond distance between hydrogens of approximately 0.163 nm, in good agreement with experimental values. The oxygen-hydrogen bond sits around 0.09 nm, also in good agreement with experiment values.

In [21]:
water_ligand.run.RadialDistributionFunction(
    number_of_configurations=5200, start=0, stop=5100, number_of_bins=500, molecules=True
)

2022-02-22 14:17:58,544 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:17:58,587 - WARNING: No GPUs detected, continuing without GPU support


Running mini batch loop 1 / 1: 100%|████| 1/1 [00:00<00:00,  3.01it/s]


Loading BokehJS ...

Exp2_Radial_Distribution_Function_4

### Diffusion Coeffients

Finally, let's start looking at the diffusion coefficients of the atoms and molecules.

In [22]:
water_chemical.run.EinsteinDiffusionCoefficients(data_range=500)

2022-02-22 14:17:59,491 - INFO: starting Einstein Diffusion Computation
2022-02-22 14:17:59,493 - INFO: starting Einstein Diffusion Computation
2022-02-22 14:17:59,654 - WARNING: No GPUs detected, continuing without GPU support


O: 100%|████████████████████████████████| 1/1 [00:06<00:00,  6.51s/it]

2022-02-22 14:18:06,372 - WARNING: No GPUs detected, continuing without GPU support



H: 100%|████████████████████████████████| 1/1 [00:07<00:00,  7.13s/it]


Loading BokehJS ...

Exp1_Einstein Self-Diffusion Coefficients_5

In [23]:
water_chemical.run.EinsteinDiffusionCoefficients(molecules=True, data_range=500)

2022-02-22 14:18:14,122 - INFO: starting Einstein Diffusion Computation
2022-02-22 14:18:14,124 - INFO: starting Einstein Diffusion Computation
2022-02-22 14:18:14,259 - WARNING: No GPUs detected, continuing without GPU support


water: 100%|████████████████████████████| 1/1 [00:06<00:00,  6.44s/it]


Loading BokehJS ...

Exp1_Einstein Self-Diffusion Coefficients_6

### Group-wise analysis

Say we want to study a specific molecule. We only want the diffusion coefficients, ADFs, and RDFs of the atoms in that one molecule. This can be achieved with the MDSuite atom-selection command and is included here as a demonstration of the flexibility of the software.

First things first, let's select a molecule group to study, say the first water molecule.

In [24]:
water_group = water_chemical.molecules['water']['groups']['0']
print(water_group)

{'H': [0, 1], 'O': [0]}


In [25]:
water_chemical.run.RadialDistributionFunction(atom_selection={'H': [0, 1], 'O': [0]}, number_of_configurations=2517)

2022-02-22 14:18:21,168 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:18:21,213 - WARNING: No GPUs detected, continuing without GPU support


Running mini batch loop 2 / 2: 100%|████| 1/1 [00:00<00:00, 34.20it/s]


Loading BokehJS ...

Exp1_Radial_Distribution_Function_7

In [26]:
water_chemical.run.AngularDistributionFunction(atom_selection=water_group, number_of_configurations=100)

2022-02-22 14:18:21,942 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:18:21,990 - WARNING: No GPUs detected, continuing without GPU support


100%|███████████████████████████████████| 1/1 [00:00<00:00, 15.27it/s]


Loading BokehJS ...

Exp1_Angular_Distribution_Function_8

In [27]:
water_chemical.run.EinsteinDiffusionCoefficients(atom_selection=water_group, data_range=2500)

2022-02-22 14:18:22,469 - INFO: starting Einstein Diffusion Computation
2022-02-22 14:18:22,470 - INFO: starting Einstein Diffusion Computation
2022-02-22 14:18:22,604 - WARNING: No GPUs detected, continuing without GPU support


O: 100%|████████████████████████████████| 1/1 [00:03<00:00,  3.60s/it]

2022-02-22 14:18:26,376 - WARNING: No GPUs detected, continuing without GPU support



H: 100%|████████████████████████████████| 1/1 [00:03<00:00,  3.77s/it]


Loading BokehJS ...

Exp1_Einstein Self-Diffusion Coefficients_9

# BMIM-BF4 RTIL

In [28]:
bmim_bf4 = DataHub(
    url="https://github.com/zincware/DataHub/tree/main/Bmim_BF4"
)
bmim_bf4.get_file()
bmim_file = bmim_bf4.file_raw

In [29]:
project.add_experiment("bmim_bf4", simulation_data=bmim_file)
project.experiments.bmim_bf4.run.CoordinateUnwrapper()

2022-02-22 14:18:33,852 - INFO: Creating a new experiment!
2022-02-22 14:18:34,948 - WARNING: No GPUs detected, continuing without GPU support


100%|███████████████████████████████████| 1/1 [00:08<00:00,  8.73s/it]


2022-02-22 14:18:44,147 - WARNING: No GPUs detected, continuing without GPU support


Applying transformation 'Unwrapped_Positions' to 'N': 100%|█| 1/1 [00:

2022-02-22 14:18:44,367 - WARNING: No GPUs detected, continuing without GPU support


...

Applying transformation 'Unwrapped_Positions' to 'H': 100%|█| 1/1 [00:

2022-02-22 14:18:45,080 - WARNING: No GPUs detected, continuing without GPU support



Applying transformation 'Unwrapped_Positions' to 'B': 100%|█| 1/1 [00:


2022-02-22 14:18:45,266 - WARNING: No GPUs detected, continuing without GPU support


Applying transformation 'Unwrapped_Positions' to 'F': 100%|█| 1/1 [00:


In [32]:
bmim_molecule = mdsuite.Molecule(
            name="bmim",
            species_dict={"C": 8, "N": 2, "H": 15},
            amount=50,
            cutoff=1.9,
            reference_configuration_idx=100,
        )
bf_molecule = mdsuite.Molecule(
    name="bf4",
    smiles="[B-](F)(F)(F)F",
    amount=50,
    cutoff=2.4,
    reference_configuration_idx=100,
)
project.experiments["bmim_bf4"].run.MolecularMap(
    molecules=[bmim_molecule, bf_molecule]
)

2022-02-22 14:19:21,520 - INFO: Building molecular graph from configuration for bmim


100%|█████████████████████████████| 1250/1250 [00:18<00:00, 68.21it/s]


2022-02-22 14:20:17,606 - INFO: Performing molecule number isomorphism test.
2022-02-22 14:20:17,607 - INFO: Amount of molecules test passed.
2022-02-22 14:20:17,607 - INFO: Performing group equality isomorphism test.
2022-02-22 14:20:17,612 - INFO: Group equality isomorphism test passed.
2022-02-22 14:20:17,676 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:20:17,976 - INFO: Mapping molecule graphs onto trajectory for bmim


11it [00:01,  5.78it/s]                                               


2022-02-22 14:20:21,225 - WARNING: No GPUs detected, continuing without GPU support


...


100%|███████████████████████████████| 250/250 [00:02<00:00, 86.67it/s]


2022-02-22 14:20:27,842 - INFO: Performing molecule number isomorphism test.
2022-02-22 14:20:27,843 - INFO: Amount of molecules test passed.
2022-02-22 14:20:27,844 - INFO: Performing group equality isomorphism test.
2022-02-22 14:20:27,845 - INFO: Group equality isomorphism test passed.
2022-02-22 14:20:27,888 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:20:27,929 - INFO: Mapping molecule graphs onto trajectory for bf4


100%|███████████████████████████████████| 1/1 [00:00<00:00,  3.63it/s]

2022-02-22 14:20:28,371 - WARNING: No GPUs detected, continuing without GPU support



Applying transformation 'Positions' to 'bf4': 100%|█| 1/1 [00:00<00:00


In [33]:
project.experiments.bmim_bf4.run.RadialDistributionFunction(
    number_of_configurations=300, number_of_bins=100
)

2022-02-22 14:20:59,038 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:20:59,092 - WARNING: No GPUs detected, continuing without GPU support


100%|███████████████████████████████| 300/300 [01:50<00:00,  2.73it/s]


Loading BokehJS ...

Exp3_Radial_Distribution_Function_10

In [34]:
project.experiments.bmim_bf4.run.RadialDistributionFunction(
    number_of_configurations=500, number_of_bins=100, molecules=True
)

2022-02-22 14:22:50,901 - WARNING: No GPUs detected, continuing without GPU support
2022-02-22 14:22:50,957 - WARNING: No GPUs detected, continuing without GPU support


Running mini batch loop 2 / 2: 100%|████| 1/1 [00:00<00:00, 14.43it/s]


Loading BokehJS ...

Exp3_Radial_Distribution_Function_11